In [30]:
from pynq import Overlay, PL

In [31]:
import time

In [32]:
PL.reset()

In [33]:
filename = "archived/mpp.bit"

In [34]:
filename

'archived/mpp.bit'

In [35]:
base = Overlay(filename, download=True) # bitstream implicitly downloaded to PL

In [36]:
def build_register(signal_defaults, default_values=None, **kwargs):
    if default_values is None:
        default_values = {}

    for arg in kwargs:
        if arg not in signal_defaults:
            raise ValueError(f"Invalid signal name: {arg}")

    register = 0

    for signal, (low_bit, high_bit) in signal_defaults.items():
        value = kwargs.get(signal, default_values.get(signal, 0))
        mask = (1 << (high_bit - low_bit + 1)) - 1
        value = (value & mask) << low_bit
        register |= value

    return register


def get_reg0(**kwargs):
    signal_defaults = {
        "flbus": (0, 0),
        "pchbus": (1, 1),
        "pclbus": (2, 2),
        "acbus": (3, 3),
        "accar": (4, 4),
        "albus": (5, 5),
        "selalu": (6, 8),
        "hcar": (9, 9),
        "lcar": (10, 10),
        "op2car": (11, 11),
        "spcar": (12, 12),
        "pccar": (13, 13),
        "pchcar": (14, 14),
        "pclcar": (15, 15),
        "seldir": (16, 17),
        "id_": (18, 19),
        "membus": (20, 20),
        "le": (21, 21),
        "mxreg": (22, 23),
        "regcar": (24, 24),
        "regbus": (25, 25),
        "selfl": (26, 26),
        "flcar": (27, 27),
        "cop10": (28, 29),
        "cop32": (30, 31),
    }
    default_values = {"le": 1}
    return build_register(signal_defaults, default_values, **kwargs)


def get_reg1(**kwargs):
    signal_defaults = {"drive_control": (0, 0)}
    return build_register(signal_defaults, **kwargs)


def get_reg2(**kwargs):
    signal_defaults = {"mclk": (0, 0)}
    return build_register(signal_defaults, **kwargs)


def get_reg3(**kwargs):
    signal_defaults = {
        "lload": (0, 0),
        "data_bus_lload": (1, 8),
        "readmemclk": (9, 9),
        "readmem_offset": (10, 25),
    }
    return build_register(signal_defaults, **kwargs)


In [37]:
def write_reg(reg_num, data):
    base.get_mclk_0.mmio.write(reg_num * 4, data)

In [38]:
def read_reg(reg_num):
    return base.linker_0.mmio.read(reg_num * 4)

In [39]:
def reset_reg(reg_num):
    reg = 0
    # get the default values!!
    if (reg_num == 0):
        reg = get_reg0()
    elif (reg_num == 1):
        reg = get_reg1()
    elif (reg_num == 2):
        reg = get_reg2()
    elif (reg_num == 3):
        reg = get_reg3()

    write_reg(reg_num, reg)

In [40]:
def reset(with_control=True):
    reset_reg(0)
    reset_reg(1)
    reset_reg(2)
    reset_reg(3)
    write_reg(1, get_reg1(drive_control=(1 if with_control else 0)))

In [41]:
PC_REG_OFFSET = 0
DATA_BUS_REG_OFFSET = 1
DIR_BUS_REG_OFFSET = 2
ACUM_REG_OFFSET = 3
OP2_REG_OFFSET = 4
SP_REG_OFFSET = 5
H_REG_OFFSET = 6
L_REG_OFFSET = 7
RI_REG_OFFSET = 8
STATE_REG_OFFSET = 9
NEXT_STATE_REG_OFFSET = 10
MEM_VALUE_READED_REG_OFFSET = 11
MXDIR_OUT_REG_OFFSET = 12
RB_REG_OFFSET = 13
RC_REG_OFFSET = 14
RD_REG_OFFSET = 15
RE_REG_OFFSET = 16
ALU_OUT_REG_OFFSET = 17
FZ_REG_OFFSET = 18
FC_REG_OFFSET = 19

In [42]:
def write_to_mem(offset, value, debug=False):
    if (debug):
        print(f"Writting to {hex(offset)} value {hex(value)}")
    l = offset & 0xFF  # Mask all but the last 8 bits
    h = (offset >> 8) & 0xFF

    write_reg(3, get_reg3(lload=1, data_bus_lload=l))
    write_reg(0, get_reg0(lcar=1))
    write_reg(2, get_reg2(mclk=1))

    reset_reg(2)
    reset_reg(0)
    reset_reg(3)
    
    write_reg(3, get_reg3(lload=1, data_bus_lload=h))
    write_reg(0, get_reg0(hcar=1))
    write_reg(2, get_reg2(mclk=1))
    
    assert(h == read_reg(H_REG_OFFSET))
    assert(l == read_reg(L_REG_OFFSET))
    
    reset_reg(2)
    reset_reg(0)
    reset_reg(3)
    
    write_reg(3, get_reg3(lload=1, data_bus_lload=value))
    assert(read_reg(DATA_BUS_REG_OFFSET) == value)
    write_reg(0, get_reg0(le=0, seldir=0b10))
    assert(read_reg(DIR_BUS_REG_OFFSET) == offset)
    write_reg(2, get_reg2(mclk=1))
    
    reset_reg(2)
    reset_reg(0)
    reset_reg(3)
    
    return True

In [43]:
def read_mem(offset):
    write_reg(3, get_reg3(readmem_offset=offset)) # the readmem_offset must be BEFORE clk
    write_reg(3, get_reg3(readmem_offset=offset, readmemclk=1)) # the readmem_offset could be not present here?
    reset_reg(3)

    write_reg(3, get_reg3(readmem_offset=offset))
    write_reg(3, get_reg3(readmem_offset=offset, readmemclk=1))
    value = read_reg(MEM_VALUE_READED_REG_OFFSET) 
 
    reset_reg(3)
    return value

In [44]:
def set_pc(pc_value):
    pcl = pc_value & 0xFF  # Mask all but the last 8 bits
    pch = (pc_value >> 8) & 0xFF

    write_reg(3, get_reg3(lload=1, data_bus_lload=pcl))
    write_reg(0, get_reg0(pclcar=1))
    write_reg(2, get_reg2(mclk=1))

    reset_reg(2)
    reset_reg(0)
    reset_reg(3)
    
    write_reg(3, get_reg3(lload=1, data_bus_lload=pch))
    write_reg(0, get_reg0(pchcar=1))
    write_reg(2, get_reg2(mclk=1))
    
    reset_reg(2)
    reset_reg(0)
    reset_reg(3)

In [45]:
def print_machine_state():
    print(f"PC: {hex(read_reg(PC_REG_OFFSET))}")
    print(f"DATA_BUS: {hex(read_reg(DATA_BUS_REG_OFFSET))}")
    print(f"DIR_BUS: {hex(read_reg(DIR_BUS_REG_OFFSET))}")
    print(f"ACUM: {hex(read_reg(ACUM_REG_OFFSET))}")
    print(f"OP2: {hex(read_reg(OP2_REG_OFFSET))}")
    print(f"SP: {hex(read_reg(SP_REG_OFFSET))}")
    print(f"H: {hex(read_reg(H_REG_OFFSET))}")
    print(f"L: {hex(read_reg(L_REG_OFFSET))}")
    print(f"RI: {hex(read_reg(RI_REG_OFFSET))}")
    print(f"STATE: {hex(read_reg(STATE_REG_OFFSET))}")
    print(f"NEXT_STATE: {hex(read_reg(NEXT_STATE_REG_OFFSET))}")
    print(f"MEM_VALUE_READED: {hex(read_reg(MEM_VALUE_READED_REG_OFFSET))}")
    print(f"MXDIR_OUT: {hex(read_reg(MXDIR_OUT_REG_OFFSET))}")
    print(f"RB: {hex(read_reg(RB_REG_OFFSET))}")
    print(f"RC: {hex(read_reg(RC_REG_OFFSET))}")
    print(f"RD: {hex(read_reg(RD_REG_OFFSET))}")
    print(f"RE: {hex(read_reg(RE_REG_OFFSET))}")
    print(f"ALU_OUT: {hex(read_reg(ALU_OUT_REG_OFFSET))}")
    print(f"FZ: {hex(read_reg(FZ_REG_OFFSET))}")
    print(f"FC: {hex(read_reg(FC_REG_OFFSET))}")
    print_mem(read_reg(MXDIR_OUT_REG_OFFSET)-5, read_reg(MXDIR_OUT_REG_OFFSET)+5)

In [46]:
def print_mem(from_offset, to_offset):
    print("|-----MEMORY-----|")
    print("------------------")
    print("| offset | value |")
    print("------------------")
    for i in range(to_offset - from_offset + 1):
        print(f"| {hex(from_offset + i)} | {hex(read_mem(from_offset + i))} |")
    print("------------------")

In [47]:
def run_cycle(print_state_after=False):
    write_reg(2, get_reg2(mclk=1))
    reset_reg(2)
    if (print_state_after):
        print_machine_state()

In [48]:
def skip_cycles(until_state=0, skip_first=False):
    limit = 10
    skip_count = 0
    while (True):
        if (read_reg(STATE_REG_OFFSET) == until_state and (not skip_first or skip_count > 0)):
            break
        run_cycle()
        skip_count += 1
        if (skip_count == limit):
            print(f"Skipped {skip_count} times but not found state {until_state}")
            break

In [49]:
def load_program(program_bytes):
    offset_to_write = 0x0000
    pc_start = 0
    
    reset(with_control=True)

    for i in range(len(program_bytes)):
        write_to_mem(offset_to_write + i, program_bytes[i])
        testIt(read_mem(offset_to_write + i) == program_bytes[i], f"Wrote {hex(program_bytes[i])} to address {hex(offset_to_write + i)}") # realmente ya esta testeado, no deberia hacer falta

    set_pc(pc_start)

    reset(with_control=False)
    skip_cycles(until_state=0)  

In [50]:
# Executing modes

def execute_instruction():
    skip_cycles(until_state=0, skip_first=True)
    
def execute_program(print_elapsed_time=True):
    limit = 100
    skip_count = 0
    fin_instruction_code = 0xFF
    
    elapsed_time = float('inf')
    
    executed_instructions = 0
    time_start = time.time()
    while (True):
        if (read_reg(RI_REG_OFFSET) == fin_instruction_code):
            elapsed_time = time.time() - time_start
            break
        execute_instruction()
        executed_instructions += 1
        if (executed_instructions == limit):
            print(f"Executed {executed_instructions} instructions reaching the limit. Are you sure your program had 0xFF for finalizing the execution")
            break
    if (print_elapsed_time):
        print(f"Executed in {elapsed_time * 1000} milliseconds")
    return elapsed_time

In [51]:
def testIt(cond, case_name=""):
    status = "✅" if cond else "❌"
    print(f"  - {case_name}{': ' if len(case_name) > 0 else ''}{status}")

In [52]:
print_machine_state()

PC: 0x0
DATA_BUS: 0x0
DIR_BUS: 0x1
ACUM: 0x0
OP2: 0x0
SP: 0x0
H: 0x0
L: 0x0
RI: 0x0
STATE: 0x0
NEXT_STATE: 0x1
MEM_VALUE_READED: 0x0
MXDIR_OUT: 0x0
RB: 0x0
RC: 0x0
RD: 0x0
RE: 0x0
ALU_OUT: 0x0
FZ: 0x0
FC: 0x0
|-----MEMORY-----|
------------------
| offset | value |
------------------
| -0x5 | 0x0 |
| -0x4 | 0x0 |
| -0x3 | 0x0 |
| -0x2 | 0x0 |
| -0x1 | 0x0 |
| 0x0 | 0x0 |
| 0x1 | 0x0 |
| 0x2 | 0x0 |
| 0x3 | 0x0 |
| 0x4 | 0x0 |
| 0x5 | 0x0 |
------------------


In [55]:
print("Test PC")

reset(with_control=True)

pc_antes = read_reg(PC_REG_OFFSET) # pc antes

write_reg(0, get_reg0(seldir=0, id_=0b10, pccar=1))
write_reg(2, get_reg2(mclk=1))

pc_despues = read_reg(PC_REG_OFFSET) # pc despues
reset_reg(2)
reset_reg(0) # 0

testIt(pc_despues == pc_antes+1, "PC despues = PC antes + 1")

Test PC
  - PC despues = PC antes + 1: ✅


In [56]:
print("Test 2OP")

reset(with_control=True)

value = 0x54
write_reg(3, get_reg3(lload=1, data_bus_lload=value))
write_reg(0, get_reg0(op2car=1))
write_reg(2, get_reg2(mclk=1))

op2_value = read_reg(OP2_REG_OFFSET)
reset_reg(2)
reset_reg(0)
reset_reg(3)

testIt(op2_value == value, "Carga 2OP")

Test 2OP
  - Carga 2OP: ✅


In [57]:
print("Test PCL y data_bus")

reset(with_control=True)

# ahora quiero ver el valor de pcl, el cual hay que publicar en bus de datos
# para publicar en bus de datos
write_reg(0, get_reg0(pclbus=1))
data_bus_pcl = read_reg(DATA_BUS_REG_OFFSET)
reset_reg(0)

testIt(data_bus_pcl == (pc_despues & 0xFF), "PCL = PC(7 downto 0)")

Test PCL y data_bus
  - PCL = PC(7 downto 0): ✅


In [58]:
print("Test AC y data_bus")

reset(with_control=True)

write_reg(0, get_reg0(accar=1, pclbus=1))
data_bus_pcl_1 = read_reg(DATA_BUS_REG_OFFSET)

write_reg(0, get_reg0(accar=1, pclbus=1))
write_reg(2, get_reg2(mclk=1))
data_bus_pcl_2 = read_reg(DATA_BUS_REG_OFFSET)

reset_reg(2)
reset_reg(0) # 0

write_reg(0, get_reg0(acbus=1))
data_bus_ac = read_reg(DATA_BUS_REG_OFFSET)

reset_reg(0) # 0

#print(f"AC = {data_bus_ac}, data_bus_pcl = {data_bus_pcl}")
    

testIt(data_bus_ac == data_bus_pcl_1 == data_bus_pcl_2, "Carga y visualización en bus datos")

Test AC y data_bus
  - Carga y visualización en bus datos: ✅


In [59]:
print("Test L")

reset(with_control=True)

write_reg(0, get_reg0(lcar=1, pclbus=1))
data_bus_pcl_1 = read_reg(DATA_BUS_REG_OFFSET) 

write_reg(0, get_reg0(lcar=1, pclbus=1))
write_reg(2, get_reg2(mclk=1))
data_bus_pcl_2 = read_reg(DATA_BUS_REG_OFFSET)

reset_reg(2)
reset_reg(0)

write_reg(0, get_reg0(seldir=2))
dir_bus_hl = read_reg(DIR_BUS_REG_OFFSET)

reset_reg(0)

#print(f"AC = {data_bus_ac}, data_bus_pcl = {data_bus_pcl}")
    
testIt(data_bus_pcl_1 == data_bus_pcl_2 == dir_bus_hl, "Carga y visualización en bus de datos")

Test L
  - Carga y visualización en bus de datos: ✅


In [60]:
print("Test GREG")

reset(with_control=True)

write_reg(0, get_reg0(cop32=0b11, mxreg=0b01, regcar=1, pclbus=1))
data_bus_pcl_1 = read_reg(DATA_BUS_REG_OFFSET) 

write_reg(0, get_reg0(cop32=0b11, mxreg=0b01, regcar=1, pclbus=1))
write_reg(2, get_reg2(mclk=1))

reset_reg(2)
reset_reg(0)

write_reg(0, get_reg0(cop32=0b11, mxreg=0b01, regbus=1))
data_bus_greg = read_reg(DATA_BUS_REG_OFFSET)

reset_reg(0)

testIt(data_bus_pcl_1 == data_bus_greg, "Carga y visualizació")

Test GREG
  - Carga y visualizació: ✅


In [61]:
print("Test Mem")

reset(with_control=True)

value_to_write = 45
write_reg(3, get_reg3(lload=1, data_bus_lload=value_to_write))
write_reg(0, get_reg0(le=0))
write_reg(2, get_reg2(mclk=1))

reset_reg(2)
reset_reg(0)
reset_reg(3)

write_reg(0, get_reg0(membus=1, le=1))
#write_reg(2, get_reg2(mclk=1)) # no hace falta del todo por el clk de la fpga
data_bus_mem = read_reg(DATA_BUS_REG_OFFSET)
reset_reg(2)
reset_reg(0)


testIt(data_bus_mem == value_to_write, "Write to mem at offset 0x0000 using data_bus")

Test Mem
  - Write to mem at offset 0x0000 using data_bus: ✅


In [62]:
print("Test Mem [Background]")

reset(with_control=True)

dirr = 0x1326
value = 0xF1
write_to_mem(dirr, value)

testIt(read_mem(dirr) == value, f"Write to {hex(dirr)} value {hex(value)}")

dirr = 0x0000
value = 0xAA
write_to_mem(dirr, value)

testIt(read_mem(dirr) == value, f"Write to {hex(dirr)} value {hex(value)}")

dirr = 0xFFFF
value = 0xAA
write_to_mem(dirr, value)

testIt(read_mem(dirr) == value, f"Write to {hex(dirr)} value {hex(value)}")

dirr = 0xFFFF+1
value = 0xAA
erroned = False
try:
    write_to_mem(dirr, value)
except:
    erroned = True

testIt(erroned, f"Error while writting to address out of bounds (> 0xFFFF)")

dirr = 0xABAB
value = 0xFF+1
erroned = False
try:
    write_to_mem(dirr, value)
except:
    erroned = True

testIt(erroned, f"Error while writting value > 0xFF")

Test Mem [Background]
  - Write to 0x1326 value 0xf1: ✅
  - Write to 0x0 value 0xaa: ✅
  - Write to 0xffff value 0xaa: ✅
  - Error while writting to address out of bounds (> 0xFFFF): ✅
  - Error while writting value > 0xFF: ✅


In [63]:
print("Test MOV FA, AC")

reset(with_control=True)

mem_bytes = [0x64, 0xFA]
offset_to_write = 0x0000

for i in range(len(mem_bytes)):
    write_to_mem(offset_to_write + i, mem_bytes[i])
    testIt(read_mem(offset_to_write + i) == mem_bytes[i], f"Wrote {hex(mem_bytes[i])} to address {hex(offset_to_write + i)}") # realmente ya esta testeado, no deberia hacer falta

# set PC to 0
set_pc(0)

reset(with_control=False)
skip_cycles(until_state=0)

###
testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0")
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 1, "State = 1") # se ha ejecutado el estado 0
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 15, "Next state = 15")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
testIt(read_reg(RI_REG_OFFSET) == mem_bytes[0], "RI = 0x64")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 15, "State = 15") # se ha ejecutado el estado 1
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 4, "Next state = 4")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 4, "State = 4") # se ha ejecutado el estado 15
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 0, "Next state = 0")
testIt(read_reg(PC_REG_OFFSET) == 2, "PC = 2")
testIt(read_reg(OP2_REG_OFFSET) == mem_bytes[1], "2OP = 0xFA")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0") # se ha ejecutado el estado 4
testIt(read_reg(ACUM_REG_OFFSET) == mem_bytes[1], "AC = 0xFA")
###

Test MOV FA, AC
  - Wrote 0x64 to address 0x0: ✅
  - Wrote 0xfa to address 0x1: ✅
  - State = 0: ✅
  - State = 1: ✅
  - Next state = 15: ✅
  - PC = 1: ✅
  - RI = 0x64: ✅
  - State = 15: ✅
  - Next state = 4: ✅
  - PC = 1: ✅
  - State = 4: ✅
  - Next state = 0: ✅
  - PC = 2: ✅
  - 2OP = 0xFA: ✅
  - State = 0: ✅
  - AC = 0xFA: ✅


In [64]:
print("Test MOV FF, RB")

reset(with_control=True)

mem_bytes = [0x60, 0xFF]
offset_to_write = 0x0000

for i in range(len(mem_bytes)):
    write_to_mem(offset_to_write + i, mem_bytes[i])
    testIt(read_mem(offset_to_write + i) == mem_bytes[i], f"Wrote {hex(mem_bytes[i])} to address {hex(offset_to_write + i)}") # realmente ya esta testeado, no deberia hacer falta

# set PC to 0
set_pc(0)

reset(with_control=False)
skip_cycles(until_state=0)

###
testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0")
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 1, "State = 1") # se ha ejecutado el estado 0
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 15, "Next state = 15")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
testIt(read_reg(RI_REG_OFFSET) == mem_bytes[0], "RI = 0x64")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 15, "State = 15") # se ha ejecutado el estado 1
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 39, "Next state = 39")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 39, "State = 39") # se ha ejecutado el estado 15
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 0, "Next state = 0")
testIt(read_reg(PC_REG_OFFSET) == 2, "PC = 2")
testIt(read_reg(OP2_REG_OFFSET) == mem_bytes[1], "2OP = 0xFA")
###

###
run_cycle()

rb_value = read_reg(RB_REG_OFFSET)

testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0") # se ha ejecutado el estado 39
testIt(rb_value == mem_bytes[1], "RB = 0xFA")
###
##################################### using the loaded RB
print()

print("Test MOV RB, RD")

reset(with_control=True)

mem_bytes = [0x08]
offset_to_write = 0x0000

for i in range(len(mem_bytes)):
    write_to_mem(offset_to_write + i, mem_bytes[i])
    testIt(read_mem(offset_to_write + i) == mem_bytes[i], f"Wrote {hex(mem_bytes[i])} to address {hex(offset_to_write + i)}") # realmente ya esta testeado, no deberia hacer falta

# set PC to 0
set_pc(0)

reset(with_control=False)
skip_cycles(until_state=0)

###
testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0")
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 1, "State = 1") # se ha ejecutado el estado 0
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 2, "Next state = 2")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
testIt(read_reg(RI_REG_OFFSET) == mem_bytes[0], "RI = 0x64")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 2, "State = 2") # se ha ejecutado el estado 1
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 3, "Next state = 3")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
###

###
run_cycle()

testIt(read_reg(STATE_REG_OFFSET) == 3, "State = 3") # se ha ejecutado el estado 2
testIt(read_reg(NEXT_STATE_REG_OFFSET) == 0, "Next state = 0")
testIt(read_reg(PC_REG_OFFSET) == 1, "PC = 1")
testIt(read_reg(OP2_REG_OFFSET) == rb_value, f"2OP = RB ({hex(rb_value)})")
###

###
run_cycle()
testIt(read_reg(STATE_REG_OFFSET) == 0, "State = 0") # se ha ejecutado el estado 3
testIt(read_reg(RD_REG_OFFSET) == rb_value, f"RD = RB ({hex(rb_value)})")
###

Test MOV FF, RB
  - Wrote 0x60 to address 0x0: ✅
  - Wrote 0xff to address 0x1: ✅
  - State = 0: ✅
  - State = 1: ✅
  - Next state = 15: ✅
  - PC = 1: ✅
  - RI = 0x64: ✅
  - State = 15: ✅
  - Next state = 39: ✅
  - PC = 1: ✅
  - State = 39: ✅
  - Next state = 0: ✅
  - PC = 2: ✅
  - 2OP = 0xFA: ✅
  - State = 0: ✅
  - RB = 0xFA: ✅

Test MOV RB, RD
  - Wrote 0x8 to address 0x0: ✅
  - State = 0: ✅
  - State = 1: ✅
  - Next state = 2: ✅
  - PC = 1: ✅
  - RI = 0x64: ✅
  - State = 2: ✅
  - Next state = 3: ✅
  - PC = 1: ✅
  - State = 3: ✅
  - Next state = 0: ✅
  - PC = 1: ✅
  - 2OP = RB (0xff): ✅
  - State = 0: ✅
  - RD = RB (0xff): ✅


In [65]:
print("Program: Sum AC until AC = 5")
load_program([0x64, 0x00, 0x4B, 0x67, 0x05, 0x72, 0x00, 0x0B, 0x74, 0x00, 0x02, 0xFF])

execute_program()
testIt(read_reg(ACUM_REG_OFFSET) == 0x5, "AC = 5")

Program: Sum AC until AC = 5
  - Wrote 0x64 to address 0x0: ✅
  - Wrote 0x0 to address 0x1: ✅
  - Wrote 0x4b to address 0x2: ✅
  - Wrote 0x67 to address 0x3: ✅
  - Wrote 0x5 to address 0x4: ✅
  - Wrote 0x72 to address 0x5: ✅
  - Wrote 0x0 to address 0x6: ✅
  - Wrote 0xb to address 0x7: ✅
  - Wrote 0x74 to address 0x8: ✅
  - Wrote 0x0 to address 0x9: ✅
  - Wrote 0x2 to address 0xa: ✅
  - Wrote 0xff to address 0xb: ✅
Executed in 19.836902618408203 milliseconds
  - AC = 5: ✅


In [66]:
print("Program: 0x5 * 0x3")
load_program([
    0x60, 0x05, 0x61, 0x03, 0xC3, 0x12, 0x34, 0x10, 0xC1, 0x11,
    0x67, 0x01, 0x72, 0x00, 0x19, 0xC2, 0x30, 0xC1, 0x11, 0x66,
    0x01, 0x41, 0x74, 0x00, 0x09, 0xC2, 0xFF
])

execute_program()
testIt(read_reg(ACUM_REG_OFFSET) == 0xF, "AC = 0xF (15)")

Program: 0x5 * 0x3
  - Wrote 0x60 to address 0x0: ✅
  - Wrote 0x5 to address 0x1: ✅
  - Wrote 0x61 to address 0x2: ✅
  - Wrote 0x3 to address 0x3: ✅
  - Wrote 0xc3 to address 0x4: ✅
  - Wrote 0x12 to address 0x5: ✅
  - Wrote 0x34 to address 0x6: ✅
  - Wrote 0x10 to address 0x7: ✅
  - Wrote 0xc1 to address 0x8: ✅
  - Wrote 0x11 to address 0x9: ✅
  - Wrote 0x67 to address 0xa: ✅
  - Wrote 0x1 to address 0xb: ✅
  - Wrote 0x72 to address 0xc: ✅
  - Wrote 0x0 to address 0xd: ✅
  - Wrote 0x19 to address 0xe: ✅
  - Wrote 0xc2 to address 0xf: ✅
  - Wrote 0x30 to address 0x10: ✅
  - Wrote 0xc1 to address 0x11: ✅
  - Wrote 0x11 to address 0x12: ✅
  - Wrote 0x66 to address 0x13: ✅
  - Wrote 0x1 to address 0x14: ✅
  - Wrote 0x41 to address 0x15: ✅
  - Wrote 0x74 to address 0x16: ✅
  - Wrote 0x0 to address 0x17: ✅
  - Wrote 0x9 to address 0x18: ✅
  - Wrote 0xc2 to address 0x19: ✅
  - Wrote 0xff to address 0x1a: ✅
Executed in 28.964519500732422 milliseconds
  - AC = 0xF (15): ✅


In [67]:
print("Program: INISP 0x0123")
load_program([
    0xC3, 0x01, 0x23, 0xFF
])

execute_program()
testIt(read_reg(SP_REG_OFFSET) == 0x0123, "SP = 0x0123")

Program: INISP 0x0123
  - Wrote 0xc3 to address 0x0: ✅
  - Wrote 0x1 to address 0x1: ✅
  - Wrote 0x23 to address 0x2: ✅
  - Wrote 0xff to address 0x3: ✅
Executed in 1.7240047454833984 milliseconds
  - SP = 0x0123: ✅


In [68]:
print("Program: CALL function D = B + C")
load_program([
    0xC3, 0x01, 0x23, 0x62, 0xFF, 0x63, 0xFA, 0x64, 0x10, 0x90,
    0x2F, 0x64, 0x31, 0x90, 0x2F, 0x64, 0x42, 0x90, 0x2F, 0x64,
    0xC5, 0x90, 0x60, 0x02, 0x61, 0x02, 0xC4, 0xFF, 0xFA, 0xFF
])

execute_program()
testIt(read_reg(RD_REG_OFFSET) == read_reg(RB_REG_OFFSET)+read_reg(RC_REG_OFFSET), "D = 0x4")

Program: CALL function D = B + C
  - Wrote 0xc3 to address 0x0: ✅
  - Wrote 0x1 to address 0x1: ✅
  - Wrote 0x23 to address 0x2: ✅
  - Wrote 0x62 to address 0x3: ✅
  - Wrote 0xff to address 0x4: ✅
  - Wrote 0x63 to address 0x5: ✅
  - Wrote 0xfa to address 0x6: ✅
  - Wrote 0x64 to address 0x7: ✅
  - Wrote 0x10 to address 0x8: ✅
  - Wrote 0x90 to address 0x9: ✅
  - Wrote 0x2f to address 0xa: ✅
  - Wrote 0x64 to address 0xb: ✅
  - Wrote 0x31 to address 0xc: ✅
  - Wrote 0x90 to address 0xd: ✅
  - Wrote 0x2f to address 0xe: ✅
  - Wrote 0x64 to address 0xf: ✅
  - Wrote 0x42 to address 0x10: ✅
  - Wrote 0x90 to address 0x11: ✅
  - Wrote 0x2f to address 0x12: ✅
  - Wrote 0x64 to address 0x13: ✅
  - Wrote 0xc5 to address 0x14: ✅
  - Wrote 0x90 to address 0x15: ✅
  - Wrote 0x60 to address 0x16: ✅
  - Wrote 0x2 to address 0x17: ✅
  - Wrote 0x61 to address 0x18: ✅
  - Wrote 0x2 to address 0x19: ✅
  - Wrote 0xc4 to address 0x1a: ✅
  - Wrote 0xff to address 0x1b: ✅
  - Wrote 0xfa to address 0x1c: ✅


In [71]:
print("Program: Search in buffer for 0x05")
load_program([
    0x62, 0x01, 0x63, 0x02, 0x64, 0x05, 0x90, 0x62, 0x00, 0x63,
    0xFE, 0xB0, 0x67, 0x05, 0x72, 0x00, 0x1F, 0x13, 0x67, 0xFF,
    0x72, 0x00, 0x1A, 0x74, 0x00, 0x1B, 0x2E, 0x2F, 0x74, 0x00,
    0x0B, 0xFF
])

execute_program()
testIt(read_reg(RD_REG_OFFSET) == 0x01, "D = 0x01")
testIt(read_reg(RE_REG_OFFSET) == 0x02, "E = 0x02")

Program: Search in buffer for 0x05
  - Wrote 0x62 to address 0x0: ✅
  - Wrote 0x1 to address 0x1: ✅
  - Wrote 0x63 to address 0x2: ✅
  - Wrote 0x2 to address 0x3: ✅
  - Wrote 0x64 to address 0x4: ✅
  - Wrote 0x5 to address 0x5: ✅
  - Wrote 0x90 to address 0x6: ✅
  - Wrote 0x62 to address 0x7: ✅
  - Wrote 0x0 to address 0x8: ✅
  - Wrote 0x63 to address 0x9: ✅
  - Wrote 0xfe to address 0xa: ✅
  - Wrote 0xb0 to address 0xb: ✅
  - Wrote 0x67 to address 0xc: ✅
  - Wrote 0x5 to address 0xd: ✅
  - Wrote 0x72 to address 0xe: ✅
  - Wrote 0x0 to address 0xf: ✅
  - Wrote 0x1f to address 0x10: ✅
  - Wrote 0x13 to address 0x11: ✅
  - Wrote 0x67 to address 0x12: ✅
  - Wrote 0xff to address 0x13: ✅
  - Wrote 0x72 to address 0x14: ✅
  - Wrote 0x0 to address 0x15: ✅
  - Wrote 0x1a to address 0x16: ✅
  - Wrote 0x74 to address 0x17: ✅
  - Wrote 0x0 to address 0x18: ✅
  - Wrote 0x1b to address 0x19: ✅
  - Wrote 0x2e to address 0x1a: ✅
  - Wrote 0x2f to address 0x1b: ✅
  - Wrote 0x74 to address 0x1c: ✅
  -